In [2]:
import requests
import json
import time

# Twitter

In [4]:
# Get BEARER_TOKEN from key.json
with open('key.json') as f:
	key = json.load(f)
	bearer_token = key['BEARER_TOKEN']

In [10]:
def get_tweets(search, num_tweets = 5):
	# Create the url
	url = 'https://api.twitter.com/1.1/search/tweets.json?result_type=popular&q='+ search + '&max_results=' + str(num_tweets)
	# Create request headers
	headers = {
		'authorization': 'Bearer ' + bearer_token,
		'user-agent': 'v2FilteredStreamPython',
	}
	# Make the request
	response = requests.get(url, headers=headers)
	return response.json()

In [13]:
wheels_json = get_tweets('wheel', 100)
doors_json = get_tweets('door', 100)

In [14]:
# Write to json file
with open('wheels.json', 'w') as f:
	json.dump(wheels_json, f)
with open('doors.json', 'w') as f:
	json.dump(doors_json, f)

# Reddit

In [31]:
# Download full posts from reddit search results
def get_reddit_data(search, num_posts = 5):
	# Create the url for the search also get the posts text
	url = 'https://www.reddit.com/search.json?q=' + search + '&sort=top&t=all&limit=' + str(num_posts)
	# Make the request
	response = requests.get(url)
	return response.json()

In [32]:
wheels_reddit = get_reddit_data('wheels', 100)
doors_reddit = get_reddit_data('doors', 100)

In [33]:
with open('wheels_submissions.json', 'w') as f:
	json.dump(wheels_reddit, f)
with open('doors_submissions.json', 'w') as f:
	json.dump(doors_reddit, f)

In [ ]:
# Get the posts text
for post in wheels_reddit:
	post_id = post['data']['id']
	# Get post
	

# Wikipedia

In [35]:
def get_wikipedia_data(search, num_posts = 5):
	# Create the url for the search also get the posts text
	url = 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=' + search + '&format=json&srlimit=' + str(num_posts)
	# Make the request
	response = requests.get(url)
	return response.json()

In [39]:
wheels_wiki = get_wikipedia_data('wheels', 100)
doors_wiki = get_wikipedia_data('doors', 100)

In [37]:
with open('wheels_wiki.json', 'w') as f:
	json.dump(wheels_wiki, f)
with open('doors_wiki.json', 'w') as f:
	json.dump(doors_wiki, f)

In [41]:
# Get the pages from wikipedia
for page in wheels_wiki['query']['search']:
	url = 'https://en.wikipedia.org/wiki/' + page['title']
	response = requests.get(url)
	with open('wheels_wiki_pages/' + page['title'] + '.html', 'w') as f:
		f.write(response.text)
	time.sleep(1)

In [43]:
for page in doors_wiki['query']['search']:
	url = 'https://en.wikipedia.org/wiki/' + page['title']
	response = requests.get(url)
	with open('doors_wiki_pages/' + page['title'] + '.html', 'w') as f:
		f.write(response.text)
	time.sleep(1)